In [52]:
!pip install SpeechRecognition

In [1]:
import os
import time
import math
import urllib
from collections import deque
import speech_recognition as sr

import audioop
import numpy as np
import librosa
import matplotlib.pyplot as plt
import noisereduce as nr
import soundfile as sf
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
import pyaudio
import wave
from scipy.io import wavfile
from IPython.display import Audio

/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
RATE = 16000
SEC = 0.5
CHUNKSIZE = int(RATE * SEC)

### Noise 녹음

In [3]:
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paFloat32, channels=1, rate=RATE, input=True, output=True, frames_per_buffer=CHUNKSIZE)

data = stream.read(CHUNKSIZE*10, exception_on_overflow=False)
noise_sample = np.frombuffer(data, dtype=np.float32)
loud_threshold = math.sqrt(abs(audioop.avg(noise_sample, 4)))
print("Loud threshold", loud_threshold)

stream.stop_stream()
stream.close()
p.terminate()

Loud threshold 9246.420658827934


In [4]:
threshold = math.sqrt(abs(audioop.avg(np.concatenate([noise_sample]), 4)))

In [5]:
sf.write(file='test.wav', data=noise_sample, samplerate=16000)

In [6]:
Audio(data=noise_sample, rate=16000)

### Streaming

In [17]:
#threshold = math.sqrt(abs(audioop.avg(noise_sample, 4)))

In [7]:
audio_buffer = [] # 원본 음성을 모음
noise_reduced_buffer = [] # 노이즈 제거된 음성을 모음
noise_queue = deque(maxlen=10) # silence 구간을 노이즈라 판단하여 최대 10개까지 모음

p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paFloat32, channels=1, rate=RATE, input=True, output=True, frames_per_buffer=CHUNKSIZE)

try:
    while(True):
        # Read chunk and load it into numpy array.
        data = stream.read(CHUNKSIZE, exception_on_overflow=False)
        audio_clip = np.frombuffer(data, dtype=np.float32)
        audio_buffer = np.concatenate((audio_buffer, audio_clip))
        
        if not noise_queue:
            current_window = audio_clip
        else:
            noise_clip = np.concatenate(noise_queue)
            current_window = nr.reduce_noise(audio_clip=audio_clip, noise_clip=noise_clip/2, verbose=False)
        loud = math.sqrt(abs(audioop.avg(audio_clip, 4)))
        if loud<threshold * 1.5:
            print('Inside Silence | threshold : %.2f | loud : %.2f'%(threshold, loud))
            noise_queue.append(current_window)
            threshold = math.sqrt(abs(audioop.avg(np.concatenate(noise_queue), 4)))
            noise_reduced_buffer = np.concatenate((noise_reduced_buffer, current_window))
        else:
            print('Inside Loud    | threshold : %.2f | loud : %.2f'%(threshold, loud))
            noise_reduced_buffer = np.concatenate((noise_reduced_buffer, current_window))
except Exception as e:
    stream.stop_stream()
    stream.close()
    p.terminate()
    pass

Inside Loud    | threshold : 9246.42 | loud : 19995.44
Inside Silence | threshold : 9246.42 | loud : 7933.41
Inside Silence | threshold : 7933.41 | loud : 8394.13
Inside Silence | threshold : 8227.41 | loud : 7794.12
Inside Silence | threshold : 6561.48 | loud : 6122.34
Inside Silence | threshold : 5536.22 | loud : 7771.75
Inside Loud    | threshold : 4745.94 | loud : 7364.60
Inside Silence | threshold : 4745.94 | loud : 5158.91
Inside Loud    | threshold : 4504.67 | loud : 8082.80
Inside Loud    | threshold : 4504.67 | loud : 8538.74
Inside Silence | threshold : 4504.67 | loud : 3745.40
Inside Silence | threshold : 4023.64 | loud : 3627.03
Inside Loud    | threshold : 3021.80 | loud : 6635.76
Inside Loud    | threshold : 3021.80 | loud : 6872.33
Inside Loud    | threshold : 3021.80 | loud : 8135.73
Inside Loud    | threshold : 3021.80 | loud : 6937.14
Inside Loud    | threshold : 3021.80 | loud : 6796.33
Inside Loud    | threshold : 3021.80 | loud : 7575.48
Inside Loud    | threshold 

KeyboardInterrupt: 

In [10]:
Audio(data=audio_buffer, rate=RATE)

In [9]:
Audio(data=noise_reduced_buffer, rate=RATE)

In [11]:
Audio(data=np.concatenate(noise_queue), rate=RATE)

In [9]:
audio = sf.write('audio.wav', audio_buffer, RATE)
nr = sf.write('noise_reduced.wav', noise_reduced_buffer, RATE)

In [10]:
def stt_google_wav(audio_fname):
    AUDIO_FILE = f"{audio_fname}"
    text = []
    # audio file을 audio source로 사용합니다
    r = sr.Recognizer()
    with sr.AudioFile(AUDIO_FILE) as source:
        audio = r.record(source)  # 전체 audio file 읽기

    # 구글 웹 음성 API로 인식하기 (하루에 제한 50회)
    try:
        print("Google Speech Recognition thinks you said : " + r.recognize_google(audio, language='ko'))
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))

In [11]:
stt_google_wav('audio.wav')

Google Speech Recognition thinks you said : 파이썬 부산 코드로 배우는 다양한 해킹 좀 더 건강한 실험 테크닉


In [12]:
stt_google_wav('noise_reduced.wav')

Google Speech Recognition thinks you said : 파이썬 부산 코드로배우는 다양한 해킹 김덕관 강한 실험 테크닉


In [ ]:
sound_buffer, rate = sf.read('../audio.wav')

In [ ]:
from pydub import AudioSegment, silence

myaudio = AudioSegment.from_wav("../noise_reduced.wav")
dBFS = myaudio.dBFS
audio = silence.detect_silence(myaudio, min_silence_len=1000, silence_thresh=dBFS-16)
audio = [((start/1000),(stop/1000)) for start,stop in audio] # milisecond to second

In [ ]:
a = sound_buffer.tobytes()

In [ ]:
AudioSegment(data=sound_buffer.tobytes(),
             sample_width=2,
             frame_rate=16000,
             channels=1)

In [ ]:
myaudio.max_dBFS

In [ ]:
silence.detect_silence??

In [ ]:
list??

In [ ]:
librosa.effects.split(y=audio_buffer, frame_length=16000, top_db=10)

## To Do

* 왜 지직거리게 파일이 저장되는지 확인 필요
* pyInt16일 때 파일이 정상적으로 녹음되는 것 같은데, 그렇게 하면 float으로 받아온 데이터들이 int처리 되면서 0으로 되어서 [ParameterError: Audio buffer is not finite everywhere 에러 뜸](https://github.com/kyungyunlee/vocalMashup/issues/1)
  <br>  → fromstring으로 읽어오는 걸로 해결
* 일단 pyFloat32로 해놓은 상태인데 음질이 지직거리게 저장됨. 
   <br> → 일단 noisereduce 제외해놓은 상태 
* 너무 짧은 단위로 저장되는 중.

* noise sample 뽑을 때 이미 노이즈가 존재하는 거면 그 기준으로 loud_threshold를 잡으면 발화 도중에 소음이 있어도 silence reign에 있다고 되므로 다른 판단 필요

In [ ]:
from IPython.display import Audio

In [ ]:
Audio(data=file, rate=RATE)

In [ ]:
Audio(data=audio_buffer, rate=RATE)

### Streaming 종료

In [ ]:
# close stream
stream.stop_stream()
stream.close()
p.terminate()

## ipywebrtc 사용

In [ ]:
from ipywebrtc import AudioRecorder, CameraStream

camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera, playing=False)

In [ ]:
recorder

## audioop

In [ ]:
CHUNK = 1000
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
THRESHOLD = 1500
SILENCE_LIMIT = 1
PREV_AUDIO = 0.5
RECORD_SECONDS = 10

In [ ]:
audioop.avg??

In [ ]:
def listen_for_speech(threshold=THRESHOLD, num_phrases=-1):
    #Open stream
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* Listening mic. ")
    audio2send = []
    cur_data = ''
    rel = RATE/CHUNK # 16 / 1초에 16번으로 쪼개서 녹음
    slid_win = deque(maxlen=int(SILENCE_LIMIT * rel)) # 1 * 16 (1sec)
    prev_audio = deque(maxlen=int(PREV_AUDIO * rel))
    started = False
    n = num_phrases
    response = []

    while (num_phrases == -1 or n > 0):
        cur_data = stream.read(CHUNK, exception_on_overflow=False)
        slid_win.append(math.sqrt(abs(audioop.avg(cur_data, 2))))
        if(sum([x > threshold for x in slid_win]) > 0):
            if(not started):
                print("Starting record of phrase")
                started = True
            audio2send.append(cur_data)
        elif (started is True):
            print("Finished")
            filename = save_speech(list(prev_audio) + audio2send, p)
            started = False
            slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
            prev_audio = deque(maxlen=int(0.5 * rel)) 
            audio2send = []
            n -= 1
            print("Listening ...")
        else:
            prev_audio.append(cur_data)

    print("* Done recording")
    stream.close()
    p.terminate()

    return response

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory.' +  directory)

In [ ]:
def save_speech(data, p):
    try:
        foldername = 'original_audio/'+str(time.strftime('%Y-%m-%d'))+'/'
        createFolder(foldername)
        filename = foldername+'original_output_'+str(int(time.strftime('%H%M%S')))
        data = b''.join(data)
        wf = wave.open(filename + '.wav', 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE) 
        wf.writeframes(data)
        wf.close()
        return filename + '.wav'
    except data is None:
        print('Stop Recording')

In [ ]:
listen_for_speech()

## 침묵 구간 기준으로

In [ ]:
THRESHOLD = 2000
SILENCE_LIMIT = 0.5 
PREV_AUDIO = 0.5
RECORD_SECONDS = 10
RATE = 16000
CHUNK = 1024 
FORMAT = pyaudio.paInt16
CHANNELS = 1

p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* Listening mic. ")
audio2send = []
cur_data = ''  
rel = RATE/CHUNK
slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
prev_audio = deque(maxlen=int(PREV_AUDIO * rel))
started = False
response = []
tmp = []
try:
    while True:
        cur_data = stream.read(CHUNK,exception_on_overflow=False)
        slid_win.append(math.sqrt(abs(audioop.avg(cur_data, 4))))
        if(sum([(x > THRESHOLD) for x in slid_win]) > 0):
            if(not started):
                print("Starting record of phrase")
                started = True
            audio2send.append(cur_data)
        elif (started is True):
            print("Finished")
            started = False
            slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
            prev_audio = deque(maxlen=int(0.5 * rel))
            tmp = np.concatenate((tmp, audio2send))
            audio2send = []

            print("Listening ...")
        else:
            prev_audio.append(cur_data)
except Exception as e:
    print(e)

print("* Done recording")
stream.close()
p.terminate()

* Listening mic. 
Starting record of phrase
Finished
Listening ...
Starting record of phrase
Finished
Listening ...
Starting record of phrase
Finished
Listening ...
Starting record of phrase
Finished
Listening ...
Starting record of phrase
Finished
Listening ...
Starting record of phrase
Finished
Listening ...
Starting record of phrase
Finished
Listening ...
Starting record of phrase
Finished
Listening ...


In [ ]:
audio = sf.write('audio.wav', audio_buffer, RATE)
nr = sf.write('noise_reduced.wav', noise_reduced_buffer, RATE